

概要
ユーザーレビューの得点を使って強調フィルタリング（メモリベース）を行う

データソース
https://erogamescape.dyndns.org/~ap2/ero/toukei_kaiseki/



事前準備

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('drive/My Drive/dev/20230424_recommend_erogame/userbase.csv', encoding='utf-8')
# matrixを生成（メモリ節約のため型を調整）
df = df.pivot(index='uid', columns='game_id', values='score').astype('Int16')
# 0埋め
df.fillna(0,inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

ゲーム数が多いのでかなり欠損値が多い。
今回は欠損値を無視する形で実装する。

### フロー
* 評価マトリクスからユーザーが評価済みのゲームを抽出する
* ユーザーが評価済みのゲームのみに絞ったマトリクスを再生成する
* 類似度を計算する

### 参考
https://techblog.gmo-ap.jp/2019/12/17/machine-learning-missing/

In [ ]:
# 評価数が多すぎるデータはカットする
def get_recommend_source(df):
  delete_rows = []
  for index, row in df.iterrows():
    count = 0
    for val in row:
      if val > 0:
        count = count + 1
    if count > 1000:
      delete_rows.append(index)
  return df.drop(delete_rows)

recommend_source_df = get_recommend_source(df)

In [ ]:
def get_recommend_game(df, recommend_source, target_user):
  # 評価済みのゲームIDを取得する
  target_games = []
  for column_name, item in df.loc[target_user].items():
    if item > 0:
      target_games.append(column_name)
  # 対象のゲームだけに絞った行列に再生成
  reccomend_base_df = recommend_source[target_games]
  # 各ユーザーとの類似度計算
  score_list = {}
  target_df = df.loc[target_user][target_games]
  for index, _ in reccomend_base_df.iterrows():
    if index == target_user:
      continue
    tmp_df = reccomend_base_df.loc[index]
    similarity = cosine_similarity(np.array([target_df,tmp_df]))
    score_list[index] = similarity[0,1]
  score_list = sorted(score_list.items(), key=lambda x:x[1], reverse=True)
  # 先頭10人の点数の高いゲームを抽出
  pickup_users = score_list[:10]
  print(pickup_users)
  recommend_games = {}
  for tmp_pickup_users in pickup_users:
    key = tmp_pickup_users[0]
    value = tmp_pickup_users[1]
    for key2, value2 in df.loc[key].items():
      if value2==0:
        continue
      if (key2 not in target_games):
        if (key2 not in recommend_games):
          recommend_games[key2] = []
        recommend_games[key2].append(value2*value)
  recommend_games_result = {}
  for key, value in recommend_games.items():
    recommend_games_result[key] = sum(value) / len(value)
  recommend_games_result = sorted(recommend_games_result.items(), key=lambda x:x[1], reverse=True)
  print(recommend_games_result[:10])

# テスト
get_recommend_game(df, recommend_source_df, 'Terawo')